# Deriving Semantic Checkers from Tests to Detect Silent Failures in Production Distributed Systems

T2C (test-to-checker) is a system tool to synthesize runtime checkers from existing test cases to detect silent semantic violations in production systems.

It works in two phases. In the offline phase, it provides a static analyzer to extract test assertions, and a dynamic framework to schedule test runs under instrumented systems and record pre-conditions. The output of offline phase is a set of runtime checker templates and a pool of extracted assertions to be executed later. In the online phase, it provides plugin checker components to load templates and assertions and activate them to detect failures. It also provides scripts to install such components to target systems so they can be deployed together.

## Setup Server to Run Our Experiment

In [ ]:
from chi import context

# During the transition period, we need to opt into the some of the 
# new python-chi functions. Otherwise the functional interface will
# return the old types.
context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

Next, we'll pick which hardware to us. The following code cell looks for nodes of type matching the `node_type` variable, and filters our ones that are reserved.

In [ ]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
    hardware.show_nodes(available_nodes)
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

### Reserve node

In order to use hardware on Chameleon, you'll need to make a reservation.

In [ ]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-T2C", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

### Create a server on the node

Next, we will launch the reserved node with an image. 

In [ ]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-T2C",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu20.04", # or use image_name
)
my_server.submit(idempotent=True)

In [ ]:
my_server.execute("ls")

### Configure networking on the node

Now, we must configure the server to use the floating IP we reserved earlier. We'll also need to wait for the networking to finish configuring, which may take a few additional minutes.

In [ ]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

## Setup Our Experiment

### Requirements
* OS and JDK:
    - T2C is developed and tested under **Ubuntu 20.04/22.04** and **JDK 8**.
    - Other systems and newer JDKs may also work. We tested a few functionalities 
      on macOS Catalina (10.15.7) and Ventura (13.5.2) but the correctness is not guaranteed.

* Git (>= 2.16.2, version control)
* Apache Maven (>= 3.6.3, for T2C compilation)
* Apache Ant (>= 1.10.9, artifact testing only, for zookeeper compilation)
* JDK8 (openjdk recommended)

### Install software on the node

Now we will install our software on the node, over SSH. 

In [ ]:
# Clone git repo with experiment source code
my_server.execute("git clone https://github.com/nazuhifu/t2c-reproduced")

# Run setup script
my_server.execute('echo "=== SETUP STARTING ==="')
my_server.execute("nohup bash t2c-reproduced/scripts/setup.sh > /dev/null 2>&1 &")
my_server.execute('echo "=== SETUP FINISHED (PROCESS LAUNCHED) ==="')

### Run Experiment

Now, we can finally run the experiment.

In [ ]:
my_server.execute('echo "=== EXPERIMENT STARTING ==="')

my_server.execute(
    "nohup bash t2c-reproduced/scripts/run_experiments.sh > /dev/null 2>&1 &"
)

my_server.execute('echo "=== EXPERIMENT LAUNCHED ==="')